**Problem Statement**

The COVID-19 Pandemic has disrupted learning for more than 56 million students in the United States. In the Spring of 2020, most states and local governments across the U.S. closed educational institutions to stop the spread of the virus. In response, schools and teachers have attempted to reach students remotely through distance learning tools and digital platforms. Until today, concerns of the exacaberting digital divide and long-term learning loss among America’s most vulnerable learners continue to grow.




**Objective:**

        Our objective is to find answers to the below questions

•	What is the picture of digital connectivity and engagement in 2020?

•	What is the effect of the COVID-19 pandemic on online and distance learning, and how might this also evolve in the future?

•	How does student engagement with different types of education technology change over the course of the pandemic?

•	How does student engagement with online learning platforms relate to different geography? Demographic context (e.g., race/ethnicity, ESL, learning disability)? Learning context? Socioeconomic status?

•	Do certain state interventions, practices or policies (e.g., stimulus, reopening, eviction moratorium) correlate with the increase or decrease online engagement?



**Introduction:**

As the world becomes increasingly interconnected, so do 
the risks we face. The COVID-19 pandemic has not stopped 
at national borders. It has affected people regardless of 
nationality, level of education, income or gender. But the same 
has not been true for its consequences, which have hit the 
most vulnerable hardest.
Education is no exception. Students from privileged 
backgrounds, supported by their parents and eager and able to 
learn, could find their way past closed school doors to alternative 
learning opportunities. Those from disadvantaged backgrounds 
often remained shut out when their schools shut down. 
This crisis has exposed the many inadequacies and inequities 
in our education systems – from access to the broadband and 
computers needed for online education, and the supportive 
environments needed to focus on learning, up to the 
misalignment between resources and needs. 

In this kernel let's analyse 3 different datasets product_info , districts_info and engagement_data and find insights on the state of digital learning in 2020

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import glob
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline

# Reading data files 

**Product information data**

The product file products_info.csv includes information about the characteristics of the top 372 products with most users in 2020. The categories listed in this file are part of LearnPlatform's product taxonomy.

**LPID** : The unique identifier of the product

**URL** : Web Link to the specific product

**ProductName** : Name of the specific product

**Provider/CompanyName** : Name of the product provider

**Sector(s)** : Sector of education where the product is used

**PrimaryEssentialFunction** : The basic function of the product. There are two layers of labels here. Products are first labeled as one of these three categories: LC = Learning & Curriculum, CM = Classroom Management, and SDO = School & District Operations. Each of these categories have multiple sub-categories with which the products were labeled

In [ ]:
products_df = pd.read_csv("../input/learnplatform-covid19-impact-on-digital-learning/products_info.csv")
products_df.head()

In [ ]:
products_df.info()

**** DISTRICT INFORMATION DATA****

**Understanding the data**
The district file districts_info.csv includes information about the characteristics of school districts, including data from NCES (2018-19), FCC (Dec 2018), and Edunomics Lab. In this data set, we removed the identifiable information about the school districts. We also used an open source tool ARX (Prasser et al. 2020) to transform several data fields and reduce the risks of re-identification. For data generalization purposes some data points are released with a range where the actual value falls under. Additionally, there are many missing data marked as 'NaN' indicating that the data was suppressed to maximize anonymization of the dataset.

**district_id** : The unique identifier of the school district

**state** : The state where the district resides in

**locale** : NCES locale classification that categorizes U.S. territory into four types of areas: City, Suburban, Town, and Rural. See Locale Boundaries User's Manual for more information.

**pct_black/hispanic **: Percentage of students in the districts identified as Black or Hispanic based on 2018-19 NCES data

**pct_free/reduced :** Percentage of students in the districts eligible for free or reduced-price lunch based on 2018-19 NCES data

**countyconnectionsratio** : ratio (residential fixed high-speed connections over 200 kbps in at least one direction/households) based on the county level data from FCC From 477 (December 2018 version). See FCC data for more information.

**pptotalraw** : Per-pupil total expenditure (sum of local and federal expenditure) from Edunomics Lab's National Education Resource Database on Schools (NERD$) project. The expenditure data are school-by-school, and we use the median value to represent the expenditure of a given school district.

In [ ]:
district_df = pd.read_csv('/kaggle/input/learnplatform-covid19-impact-on-digital-learning/districts_info.csv')


In [ ]:
district_df.head()

In [ ]:
district_df.info()

# Engagement data
The engagement data are aggregated at school district level, and each file in the folder engagement_data represents data from one school district*.

📝The 4-digit file name represents district_id which can be used to link to district information in district_info.csv.

📝The lp_id can be used to link to product information in product_info.csv.


**time**	date in "YYYY-MM-DD"

**lp_id**	The unique identifier of the product

**pct_access**	Percentage of students in the district have at least one page-load event of a given product and on a given day

**engagement_index**	Total page-load events per one thousand students of a given product and on a given day


In [ ]:
path = '../input/learnplatform-covid19-impact-on-digital-learning/engagement_data'
all_files = glob.glob(path+ "/*.csv")

li=[]

for file in all_files:
    df=pd.read_csv(file,index_col=None,header=0)
    district_id = file.split('/')[4].split(".")[0]
    df['district_id']=district_id
    li.append(df)
    
engagement_df=pd.concat(li)
engagement_df=engagement_df.reset_index(drop=True)


In [ ]:
engagement_df.head()

In [ ]:
engagement_df.info()

# Missing Values Identification and Imputation:

In [ ]:
#Function to find the percent of missing values 
def missing_values(df):
    #Missing Values in dataframe
    mis_val = df.isnull().sum()
    #Missing value Percent :
    mis_val_percent = 100* df.isnull().sum() / len(df)
    #Missing Values Table
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_ren = mis_val_table.rename(
            columns = {0 : 'Missing Values', 1 : '% of Missing Values'})
    mis_val_final=mis_val_ren.sort_values('% of Missing Values',ascending=False)
    
    return mis_val_final

In [ ]:
#Product table missing values
msno.bar(products_df,color='green', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()

missing_values(products_df)

In [ ]:
#District table missing values
msno.bar(district_df,color='red', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()
missing_values(district_df)

In [ ]:
#Engagement data missing values
msno.bar(engagement_df,color='blue', sort="ascending", figsize=(10,5), fontsize=12)
plt.show()

missing_values(engagement_df)

In [ ]:
#Missisng values imputation:
#In district dataframe we have certain rows where all the columns are missing values except district id . So we shall remove such rows
districts_df=district_df.copy()

In [ ]:
districts_df = districts_df.dropna(thresh=2)

In [ ]:
#Check if there are any duplicates in dataframes
print("Number of Duplicate value in Districts Dataframe:",districts_df.duplicated().sum())
print("Number of Duplicate value in Engagement Dataframe:",engagement_df.duplicated().sum())
print("Number of Duplicate value in Products Dataframe:",products_df.duplicated().sum())


In [ ]:
#Lets drop the duplicate values in Products_df
products_df= products_df.dropna()

#Now we have droppped all the rows with missing values. 
missing_values(products_df)


In [ ]:
#In the description it is given PrimaryEssentialFunction has 2 layers. So now lets split those 2 layers
products_df['PEF L1'] = products_df['Primary Essential Function'].apply(lambda pef:pef.split('-')[0].strip())
products_df['PEF L2'] = products_df['Primary Essential Function'].apply(lambda pef:pef.split('-')[1].strip())
products_df = products_df[['LP ID','URL','Product Name','Provider/Company Name','Sector(s)','PEF L1','PEF L2']]
products_df.info()

In [ ]:
products_df.head()

In [ ]:
#Companies selling more than 1 product
provider_group = products_df.groupby('Provider/Company Name')[['LP ID']].count().sort_values(by='LP ID')
provider_group = provider_group[provider_group['LP ID']>1]
provider_group


In [ ]:
plt.figure(figsize=(12,6))
plt.title("Number of product by Provider/Company Name'")
plt.ylabel('Provider Name')
plt.xlabel('Number of product')
sns.barplot(x=list(provider_group['LP ID']),y=provider_group.index)


In [ ]:
#Number of products by Primary Essential function L1
pef_group = products_df.groupby('PEF L1')[['LP ID']].count().sort_values(by='LP ID')
pef_group

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Number of product by PEF L1'")
plt.ylabel('PEF L1')
plt.xlabel('Number of product')
sns.barplot(x=list(pef_group['LP ID']),y=pef_group.index)


In [ ]:
plt.figure(figsize=(16, 20))
sns.countplot(y='PEF L2', data=products_df, order=products_df["PEF L2"].value_counts().index)
plt.title("Primary Essential Function L2",font="Serif", size=20)
plt.show()


In [ ]:
#Number of products by sector
sector_group = products_df.groupby('Sector(s)')[['LP ID']].count().sort_values(by='LP ID')
sector_group

In [ ]:
plt.figure(figsize=(12,6))
plt.title("Number of product by Sector")
plt.ylabel('Sector Name')
plt.xlabel('Number of product')
sns.barplot(x=list(sector_group['LP ID']),y=sector_group.index)


In [ ]:
# Only one  company each is dealing with Higher ed and corporate sector. So lets find what are they 
products_df[products_df['Sector(s)']=='Higher Ed; Corporate']['Product Name'],products_df[products_df['Sector(s)']=='Corporate']['Product Name']

In [ ]:
#Sector Distribution
sec_colors = ['gold', 'mediumturquoise', 'darkorange','red','green']
labels = list(products_df['Sector(s)'].value_counts().index)
values = products_df['Sector(s)'].value_counts()
# colors = ['mediumslateblue', 'darkorange']
fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', textfont_size=10,
                  marker=dict(colors=sec_colors, line=dict(color='#000000', width=3)))
fig.update_layout(title="Sector Distribution ",
                  titlefont={'size': 20},      
                  )
fig.show()

In [ ]:
# State Level Data
plt.figure(figsize=(16, 10))
sns.countplot(y="state",data=districts_df,order=districts_df.state.value_counts().index,palette="Blues",linewidth=3)
plt.title("State level data",font="Serif", size=20,pad=20)
plt.show()


states = districts_df.groupby(by ='state').count()[['district_id']]
#abbreviations of all the US States
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District Of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

codes = []
for i in states.index:
    codes.append(us_state_abbrev[i])
# print(codes)
data = dict(
        type = 'choropleth',
        colorscale = 'blues',
        locations = codes,
        locationmode = 'USA-states',
        z = list(states['district_id']),
        text = states.index,
        colorbar = {'title':'States'},
      )
layout = dict(title = 'States',
              geo = dict(projection = {'type':'mercator'})
             )
layout = dict(title= 'States with most districts mentioned',
              geo = {'scope':'usa'})

choromap = go.Figure(data = [data],layout = layout)
iplot(choromap)


In [ ]:
# State Distribution
labels = list(districts_df.state.value_counts().index)
values = districts_df['state'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values,hole=.2)])
fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=15,
                  marker=dict( line=dict(color='#000000', width=3)))
fig.update_layout(title="State Distribution ",
                  titlefont={'size': 30},      
                  )
fig.show()

In [ ]:
#Locale Distribution
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
labels = list(districts_df.locale.value_counts().index)
values = districts_df['locale'].value_counts()

fig = go.Figure(data=[go.Pie(labels=labels,
                             values=values)])
fig.update_traces(hoverinfo='label+percent', textinfo='label+percent', textfont_size=20,
                  marker=dict(colors=colors,line=dict(color='#000000', width=2)))
fig.update_layout(title="Locale Distribution ",
                  titlefont={'size': 20},      
                  )
fig.show()

In [ ]:
district_df['pp_total_raw'].value_counts()

In [ ]:
district_df["pp_total_raw"].value_counts().head(10).plot(kind = 'pie', autopct='%1.1f%%',figsize=(10, 10)).legend()

In [ ]:
districts_df['pct_free/reduced'].value_counts()

In [ ]:
plt.figure(figsize=(16,7))
sns.countplot(x='pct_free/reduced',data=district_df,hue='locale')
plt.title('pct_free/reduced by the locale')

In [ ]:
# Engagement Data 
engagement_df['date'] = pd.to_datetime(engagement_df['time']).dt.date
engagement_df['month']= pd.to_datetime(engagement_df['time']).dt.month_name()
engagement_df['weekday']= pd.to_datetime(engagement_df['time']).dt.day_name()

In [ ]:
engagement_df

In [ ]:
#Monthly percentage access
plt.figure(figsize=(15,12))
sns.set_style('whitegrid')
sns.stripplot(x="month", y="pct_access", data=engagement_df)
plt.show()

In [ ]:
#Weekly percentage access
order = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
plt.figure(figsize=(15,12))
sns.set_style('whitegrid')
sns.stripplot(x="weekday", y="pct_access", data=engagement_df,order=order)
plt.show()